In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [4]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(Oh, dear, !)",Carroll


In [7]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [ ]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

In [ ]:
rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

In [ ]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

In [ ]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

In [ ]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

In [ ]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [ ]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [ ]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

In [ ]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)

Well look at that! NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland. Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

## Challenge 0:
Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.



In [ ]:
# Only slighly altered the functions above to keep the punctuation
def words_punct(text):
    
    # Filter out stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_stop
                ]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
            df.pos = 0
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords_punct = words_punct(alice_doc)
persuasionwords_punct = words_punct(persuasion_doc)

# Combine bags to create a set of unique words.
common_words_punct = set(alicewords_punct + persuasionwords_punct)

# Create data frame with features. This can take a while
word_counts_punct = bow_features(sentences, common_words_punct)

In [ ]:
word_counts_punct['punct_ratio'] = 0
word_counts_punct['noun_ratio'] = 0
word_counts_punct['verb_ratio'] = 0
word_counts_punct['adj_ratio'] = 0
word_counts_punct['sents_len'] = 0
word_counts_punct['stop_ratio'] = 0
word_counts_punct['word_len'] = 0
def new_features(df):
    for i, sentence in enumerate(df['text_sentence']):
        punct_count, noun_count, verb_count, adj_count, stop_count, char_count = 0, 0, 0, 0, 0, 0
        for word in df.loc[i, 'text_sentence']:
            punct_count += word.is_punct
            noun_count += (word.pos_=='NOUN')
            verb_count += (word.pos_=='VERB')
            adj_count += (word.pos_=='ADJ')
            stop_count += word.is_stop
        df.loc[i, 'punct_ratio'] = punct_count/len(sentence)
        df.loc[i, 'noun_ratio'] = noun_count/len(sentence)
        df.loc[i, 'verb_ratio'] = verb_count/len(sentence)
        df.loc[i, 'adj_ratio'] = adj_count/len(sentence)
        df.loc[i, 'stop_ratio'] = stop_count/len(sentence)
        df.loc[i, 'sents_len'] = len(sentence)
        df.loc[i, 'word_len'] = len(str(sentence))/len(sentence)

        if i % 500 == 0:
            print(i)

In [ ]:
new_features(word_counts_punct)

In [ ]:
word_counts_punct.head()

In [ ]:
# Test new features using best model from above (logistic)# Test n 

Y = word_counts_punct['text_source']
X = word_counts_punct.drop(['text_sentence', 'text_source'], 1)

# Split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

lr = LogisticRegression()
lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

In [ ]:
lr_params = {'C':np.arange(0.00001, 3, .01)}

In [ ]:
lr = LogisticRegression()
gs_lr = GridSearchCV(lr, lr_params, return_train_score=True, n_jobs =-1)
gs_lr.fit(X_train,y_train)

print(gs_lr.best_params_)

In [ ]:
lr = LogisticRegression(C =1.96001 )
lr.fit(X_train, y_train)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

Keeping the punctuation improved the accuracy by around 4% from 91.6 and the overfitting is just slightly less, but the overfitting wasn't really an issue before.

In [ ]:
svc = SVC()
svc.fit(X_train, y_train)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

SVC did not do quite as good as logistic regression, but I thought the test score getting a better accuracy than the training was odd.

## Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [ ]:

# adding a new author and new book to the mix
hamlet = gutenberg.raw('shakespeare-hamlet.txt')

In [ ]:
hamlet = text_cleaner(hamlet)

In [ ]:
hamlet = re.sub(r'Chapter \d+', '', hamlet)

In [ ]:
hamlet_doc = nlp(hamlet)
hamlet_sents = [[sent, 'Shakespeare'] for sent in hamlet_doc.sents]
ham_ali_sentences = pd.DataFrame(alice_sents + hamlet_sents)
ham_per_sentences = pd.DataFrame(persuasion_sents + hamlet_sents)

In [ ]:
hamwords_punct = words_punct(hamlet_doc)
common_words_punct1 = set(alicewords_punct + hamwords_punct)
word_counts_punct1 = bow_features(ham_ali_sentences, common_words_punct1)

In [ ]:
common_words_punct2 = set(persuasionwords_punct + hamwords_punct)
word_counts_punct2 = bow_features(ham_per_sentences, common_words_punct2)

In [ ]:
word_counts_punct1['punct_ratio'] = 0
word_counts_punct1['noun_ratio'] = 0
word_counts_punct1['verb_ratio'] = 0
word_counts_punct1['adj_ratio'] = 0
word_counts_punct1['sents_len'] = 0
word_counts_punct1['stop_ratio'] = 0
word_counts_punct1['word_len'] = 0

word_counts_punct2['punct_ratio'] = 0
word_counts_punct2['noun_ratio'] = 0
word_counts_punct2['verb_ratio'] = 0
word_counts_punct2['adj_ratio'] = 0
word_counts_punct2['sents_len'] = 0
word_counts_punct2['stop_ratio'] = 0
word_counts_punct2['word_len'] = 0

new_features(word_counts_punct1)
new_features(word_counts_punct2)

The real test, whether this model works when other texts is added to the mix

In [ ]:
Y = word_counts_punct1['text_source']
X = np.array(word_counts_punct1.drop(['text_sentence','text_source'], 1))
print('Scores:', cross_val_score(lr, X, Y, cv=5))

In [ ]:
Y = word_counts_punct2['text_source']
X = np.array(word_counts_punct2.drop(['text_sentence','text_source'], 1))
print('Scores:', cross_val_score(lr, X, Y, cv=5))

## Conclusion
My logistic regression model did very well in all the tests. Even before I added new features and kept the punctuation the model was performing well. Tuning the logistic regression model further barely changed the accuracy. The new results were still very close to the original results' accuracy the second test actually gave better results than the original, I am not sure one could ask for better results.